In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import random
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
#import cmsisdsp as dsp
import random

import sys
sys.path.append('/home/mat/dev/cached_dilated_causal_convolutions/') 

from cmsisdsp_py_version.block import Block
from cmsisdsp_py_version.keras_model import create_dilated_model
from cmsisdsp_py_version.cached_block_model import CachedBlockModel, Classifier
from cmsisdsp_py_version.rolling_cache import RollingCache


In [ ]:
def parse(fname):
    df_w = pd.read_csv(fname, sep=' ', names=['in', 'out_l', 'out_r'])
    df_w['n'] = range(len(df_w))
    df_l = df_w.melt(id_vars='n', value_vars=['in', 'out_l', 'out_r'])
    return df_w, df_l

sine_df_w, sine_df_l = parse('../datalogger_firmware/sine.ssv')
saw_df_w, saw_df_l = parse('../datalogger_firmware/saw.ssv')
square_df_w, square_df_l = parse('../datalogger_firmware/square.ssv')

In [ ]:
plt.figure(figsize=(16, 6))
offset = 5500
width = 3000
df = sine_df_l
window = (df['n']>offset) & (df['n']<offset+width)
sns.lineplot(df[window], x='n', y='value', hue='variable')

In [ ]:
plt.figure(figsize=(16, 6))
offset = 100
width = 2000
df = saw_df_l
window = (df['n']>offset) & (df['n']<offset+width)
sns.lineplot(df[window], x='n', y='value', hue='variable')

some super weird stuff going on with the square wave :)

In [ ]:
plt.figure(figsize=(16, 6))
offset = 4000
width = 1000
df = square_df_l
window = (df['n']>offset) & (df['n']<offset+width)
sns.lineplot(df[window], x='n', y='value', hue='variable')

training with sine worked, but the wave was boring
let's try the square next

In [ ]:
def splits(df_w):
    data = df_w.to_numpy()
    assert data.shape[1] == 4  # 4 columns
    x = data[:,0:1]
    y_true = data[:,1:3]

    split = int(len(x) * 0.05)
    train_x, train_y = x[:(2*-split)], y_true[:(2*-split)]
    validate_x, validate_y = x[(2*-split):-split], y_true[(2*-split):-split]
    test_x, test_y = x[-split:], y_true[-split:]

    return {'train_x': train_x, 'train_y': train_y, 
            'validate_x': validate_x, 'validate_y': validate_y,
            'test_x': test_x, 'test_y': test_y }

sine_splits = splits(sine_df_w)
saw_splits = splits(saw_df_w)
square_splits = splits(square_df_w)


In [ ]:
# mono in -> stereo out
IN_D = 1
OUT_D = 2

# kernel size and implied dilation rate
K = 4                

# filters for Nth layer Kx1 and 1x1 convs
# [4, 8, 8, 8] @ 32kHz => 82%
# [4, 8, 8, 12] @ 32kHz => 93%
FILTER_SIZES = [4, 8, 8, 12]  

RECEPTIVE_FIELD_SIZE = K**len(FILTER_SIZES)

TEST_SEQ_LEN = RECEPTIVE_FIELD_SIZE
TRAIN_SEQ_LEN = RECEPTIVE_FIELD_SIZE * 10

print("RECEPTIVE_FIELD_SIZE", RECEPTIVE_FIELD_SIZE)
print("TRAIN_SEQ_LEN", TRAIN_SEQ_LEN)
print("TEST_SEQ_LEN", TEST_SEQ_LEN)

In [ ]:
def create_configured_keras_model(seq_len, all_outputs):
    return create_dilated_model(
        seq_len, in_d=IN_D, filter_sizes=FILTER_SIZES,
        kernel_size=K, out_d=OUT_D,
        all_outputs=all_outputs)

In [ ]:
train_model = create_configured_keras_model(TRAIN_SEQ_LEN, all_outputs=False)

we want to ignore the first RECEPTIVE_FIELD_SIZE elements of the loss since they have been fed with left padded data

In [ ]:
mask = np.concatenate([np.zeros(RECEPTIVE_FIELD_SIZE), 
                       np.ones(TRAIN_SEQ_LEN-RECEPTIVE_FIELD_SIZE)])
mask = tf.convert_to_tensor(mask.astype(np.float32))

def masked_mse(y_true, y_pred):    
    mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)
    return tf.multiply(mse, mask)    


In [ ]:
from tensorflow.keras.optimizers import Adam

# which splits to use for train/validate
splits = sine_splits

def dataset_from(x, y):
    def gen():
        # TODO: could shuffle x and y here by indexes
        for i in range(len(x)-TRAIN_SEQ_LEN-1):
            yield x[i:i+TRAIN_SEQ_LEN], y[i+1:i+1+TRAIN_SEQ_LEN]
                
    ds = tf.data.Dataset.from_generator(
        gen, output_signature=(tf.TensorSpec(shape=(TRAIN_SEQ_LEN, IN_D), dtype=tf.float32),
                               tf.TensorSpec(shape=(TRAIN_SEQ_LEN, OUT_D), dtype=tf.float32)))
    return ds

train_ds = tf.data.Dataset.sample_from_datasets([
    dataset_from(sine_splits['train_x'], sine_splits['train_y']).cache().shuffle(10000),
    dataset_from(saw_splits['train_x'], saw_splits['train_y']).cache().shuffle(10000),
    dataset_from(square_splits['train_x'], square_splits['train_y']).cache().shuffle(10000),
])
train_ds = train_ds.batch(128).prefetch(tf.data.AUTOTUNE)

# train_ds = dataset_from(splits['train_x'], splits['train_y'])
# train_ds = train_ds.cache().shuffle(10000).batch(128).prefetch(tf.data.AUTOTUNE)

def scheduler(epoch, lr):    
    return 1e-4 if epoch == 1 else 1e-5    
sch_callback = tensorflow.keras.callbacks.LearningRateScheduler(scheduler)

validate_ds = dataset_from(splits['validate_x'], splits['validate_y'])
validate_ds = validate_ds.batch(128).prefetch(tf.data.AUTOTUNE)

train_model.compile(Adam(1e-4), loss=masked_mse)
train_model.fit(train_ds, 
                validation_data=validate_ds,
                callbacks=[sch_callback],
                epochs=3)

In [ ]:
train_model.save("trained_models/all_three")

# check against test data


In [ ]:
test_records = []

# which split to use for test
splits = sine_splits

for i in range(2000, 3000):
    test_seq = splits['test_x'][i:i+TRAIN_SEQ_LEN]
    test_seq = np.expand_dims(test_seq, 0)  # single element batch
    
    y_true = splits['test_y'][i+TRAIN_SEQ_LEN]
            
    y_pred = train_model(test_seq).numpy().squeeze()
    y_pred = y_pred[-1]  # train model gives all steps, we just want last
    
    for c in range(2):
        test_records.append((i, c, 'test_x', test_seq[0,-1,0])) 
        test_records.append((i, c, 'y_true', y_true[c]))
        test_records.append((i, c, 'y_pred', y_pred[c]))
    
test_df = pd.DataFrame(test_records, columns=['n', 'c', 'name', 'value'])

plt.figure(figsize=(16, 6))
sns.lineplot(data=test_df[test_df['c']==1], x='n', y='value', hue='name')

## generate code

hacktastically generate some blocks of c++ code for the inference

In [ ]:
assert len(train_model.layers) == 10, len(train_model.layers)

# layer[0] is input

blocks = [
    Block(
        c1_kernel = train_model.layers[1].weights[0].numpy(),
        c1_bias = train_model.layers[1].weights[1].numpy(),
        c2_kernel = train_model.layers[2].weights[0].numpy(),
        c2_bias = train_model.layers[2].weights[1].numpy(),
    ),
    Block(
        c1_kernel = train_model.layers[3].weights[0].numpy(),
        c1_bias = train_model.layers[3].weights[1].numpy(),
        c2_kernel = train_model.layers[4].weights[0].numpy(),
        c2_bias = train_model.layers[4].weights[1].numpy(),
    ),
    Block(
        c1_kernel = train_model.layers[5].weights[0].numpy(),
        c1_bias = train_model.layers[5].weights[1].numpy(),
        c2_kernel = train_model.layers[6].weights[0].numpy(),
        c2_bias = train_model.layers[6].weights[1].numpy(),
    ),
    Block(
        c1_kernel = train_model.layers[7].weights[0].numpy(),
        c1_bias = train_model.layers[7].weights[1].numpy(),
        c2_kernel = train_model.layers[8].weights[0].numpy(),
        c2_bias = train_model.layers[8].weights[1].numpy(),
    )
]

classifier = Classifier(
    weights=train_model.layers[9].weights[0].numpy()[0],
    biases=train_model.layers[9].weights[1].numpy()   
)

# create CachedBlockModel since it creates correct layer
# caches
cached_block_model = CachedBlockModel(
    blocks=blocks,
    input_feature_depth=IN_D,
    classifier=classifier
)      

with open("/tmp/model_defn.h", 'w') as f:
    cached_block_model.write_model_defn_h(f) #sys.stdout)
